In [14]:
#CONTENTS OF trainingexamples.csv

#Rainy,Cold,High,Strong,Warm,Change,N
#Sunny,Warm,Normal,Strong,Warm,Same,Y
#Sunny,Warm,High,Strong,Warm,Same,Y
#Sunny,Warm,High,Strong,Cool,Change,Y

In [15]:
import random  #not actually needed
import csv as csv    #to open and read csv file

In [16]:
# FIRST OPEN THE TRAINING DATA SET
# STORE IT AS "LIST OF TUPLES"
# EACH TUPLE IS EACH ROW IN DATA SET
with open("trainingexamples.csv") as csvfile:
    examples=[tuple(line) for line in csv.reader(csvfile)]
    #print(examples)

In [17]:
# THIS FUNCTION RETURNS A LIST CONTAINING 7(TOT NO.OF COLS) SETS
# EACH SET CONTAINING POSSIBLE VALUES OF EACH COLUMN SORTED
# THE POSSIBLE VALUES ARE SORTED IN ALPHABETICAL ORDER
# CALLED FROM E2
def get_domains(examples):
    d=[set() for i in examples[0]]
    #print(d)
    for x in examples: # ACCESS EVERY ROW
        for i,xi in enumerate(x): #i is iterator created by enumerate funcn.
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

#get_domains(examples)

In [18]:
#E3 CHECKING IF h1 IS CONSISTENT WITH h2
def consistent(h1,h2):
    mgp=[]  #zip function
    for x,y in zip(h1,h2):
        mg= x=="?" or(x!="ɸ" and (x==y or y=="ɸ"))  #mg is BOOLEAN VALUE
        mgp.append(mg)
    return all(mgp)  #all method returns TRUE if all items in all(ITER_OBJECT) are true
                                    #also it returns TRUE if ITER_OBJECT is empty

In [19]:
# RETURNS CONSISTENT MINIMAL GENERALIZATION OF GIVEN HYPOTHESIS (E4)
def min_generalizations(h,x):
    h_new=list(h)
    for i in range(len(h)):
        if not consistent(h[i:i+1],x[i:i+1]):    #WE WANT BOTH PARAMETERS TO BE ITERABLES (LIST),TATS WHY h[i:i+1]
            h_new[i]="?" if h[i]!="ɸ" else x[i]  # IF CONSISTENT, WE are making it x[i], otherwise "?"
    return [tuple(h_new)]
        

In [20]:
# RETURNS CONSISTENT MINIMAL SPECIALIZATION OF GIVEN HYPOTHESIS (E4)
def min_specializations(h,domains,x):
    results=[]
    for i in range(len(h)):
        if h[i]=="?":
            for val in domains[i]:
                if x[i]!=val:  #x[i] is leading to NEGATIVE TARGET, SO WE DONT WANT val matching with it
                    h_new=h[:i]+(val,)+h[i+1:]  #WE ARE SPECIALIZING TAT PARTICULAR ATTRIBUTE 
                    results.append(h_new)
        elif h[i]!="ɸ":
            h_new=h[:i]+("ɸ",)+h[i+1:] #IF IT IS ɸ, NOTHING TO DO, JUST COPY AS IT IS
            results.append(h_new)
    return results
                

In [21]:
# GENERALIZE THE S(SPECIFIC HYPOTHESIS) (E3)
def generalize_S(x,G,S):
    S_prev=list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not consistent(s,x):             #IF NOT CONSISTENT, THEN REMOVE
            S.remove(s)                     # ADD TO S, all MINIMAL GENERALIZATIONS TAT ARE CONSISTENT WITH x
            Splus=min_generalizations(s,x)  #REMOVE OLDER HYPOTHESIS(WHICH WERE CONSISTENT BUT MORE GENERAL)
            
            S.update([h for h in Splus if any([consistent(g,h) for g in G])])
            S.difference_update([h for h in S if any([consistent(h,h1) for h1 in S if h!=h1])]) #CiCn
    return S

In [22]:
#SPECIALIZE THE G(GENERAL HYPOTHESIS) (E3)
def specialize_G(x,domains,G,S):
    G_prev=list(G)
    for g in G_prev:
        if g not in G:
            continue
        if consistent(g,x): #NOW x IS LEADING TO NEGATIVE TARGET, SO IF SOME g IS CONSISTENT WITH x, remove it
            G.remove(g)
            Gminus=min_specializations(g,domains,x)
            
            G.update([h for h in Gminus if any([consistent(h,s) for s in S])])   #CiCn
            G.difference_update([h for h in G if any([consistent(h,g1) for g1 in G if h!=g1])]) #Cicn
    return G
        

In [23]:
#E2
def candidate_elimination(examples):
    domains=get_domains(examples)[:-1]
    #NOW LAST COLUMN(TARGET) IS NOT IN domains
    #INITIALZE G TO ?, S TO ɸ
    G = set([("?",)*len(domains)])
    S = set([("ɸ",)*len(domains)])
    
    i=0
    print("\n G[{0}]:".format(i),G)
    print("\n S[{0}]:".format(i),S) #PRINTING INITIAL G,S
    
    for xcx in examples: #EXAMINE ALL ROWS
        i=i+1
        #x is storing list of all values in the row
        #cx is storing target value of the row
        x,cx=xcx[:-1],xcx[-1]  
        if cx=="Y":
            G={g for g in G if consistent(g,x)}
            S=generalize_S(x,G,S)
        else:
            S={s for s in S if not consistent(s,x)}
            G=specialize_G(x,domains,G,S)
        print("\n G[{0}]:".format(i),G)
        print("\n S[{0}]:".format(i),S)
    return   
        
        
    

In [24]:
#EXECUTION STARTS E1
candidate_elimination(examples)


 G[0]: {('?', '?', '?', '?', '?', '?')}

 S[0]: {('ɸ', 'ɸ', 'ɸ', 'ɸ', 'ɸ', 'ɸ')}

 G[1]: {('Sunny', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same'), ('?', 'Warm', '?', '?', '?', '?'), ('?', '?', 'Normal', '?', '?', '?'), ('?', '?', '?', '?', 'Cool', '?')}

 S[1]: {('ɸ', 'ɸ', 'ɸ', 'ɸ', 'ɸ', 'ɸ')}

 G[2]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same'), ('?', '?', 'Normal', '?', '?', '?')}

 S[2]: {('Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same')}

 G[3]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', 'Same')}

 S[3]: {('Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same')}

 G[4]: {('?', 'Warm', '?', '?', '?', '?'), ('Sunny', '?', '?', '?', '?', '?')}

 S[4]: {('Sunny', 'Warm', '?', 'Strong', '?', '?')}
